# Notebook 3 - Comparing RGB values to RGB composite loop.

To better understand what RGB thresholds compare to specific types of clouds, we can plot their values next to the composite map.

In [23]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import imageio
import pandas as pd
import os

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore", message="Engine 'cfgrib' loading failed:")

#### Using different version of the `plot_rgb_image` function because we're adding another pannel to the plot

In [45]:
# Function to generate the time string based on date and time of day
def generate_time(date, time_of_day):
    # Convert date string to datetime object
    date_obj = datetime.strptime(date, '%Y%m%d')
    # Format the date part
    date_str = date_obj.strftime('%Y-%m-%d')
    # Combine date and time of day
    time_str = f'{date_str}T{time_of_day}'
    return time_str

def plot_rgb_image_and_timeseries(ds, timeseries_ds, date, time_of_day, gif=False):
    """""
    Plot RGB image from xarray dataset and save as PNG.
    Parameters:
    ds (xarray.Dataset): The dataset containing the RGB channels.
    timeseries_ds (xarray.Dataset): The dataset containing the timeseries data for the RGB channels.
    date (str): The date in 'YYYYMMDD' format.
    time_of_day (str): The time of day in 'HH:MM:SS' format.
    gif (bool): Whether to save the plot as a GIF.
    """

    # Extract the values as NumPy arrays
    red = ds['red'].values
    green = ds['green'].values
    blue = ds['blue'].values

    # Find the minimum shape among the arrays
    min_shape = np.min([red.shape, green.shape, blue.shape], axis=0)

    # Resize the arrays to the minimum shape
    red_resized = red[:min_shape[0], :min_shape[1]]
    green_resized = green[:min_shape[0], :min_shape[1]]
    blue_resized = blue[:min_shape[0], :min_shape[1]]

    # Ensure the arrays have the same dimensions
    assert red_resized.shape == green_resized.shape == blue_resized.shape, "Arrays must have the same shape"

    # Stack the arrays along the last dimension to create an RGB image
    rgb_image = np.stack([red_resized, green_resized, blue_resized], axis=-1)

    # Extract longitude and latitude values
    lon = ds['longitude'].values
    lat = ds['latitude'].values

    # Plot the RGB image using matplotlib's imshow
    fig, ax = plt.subplots(2,1,figsize=(10, 10))

    # Plot the RGB image
    rgb_plot = ax[0].imshow(rgb_image, extent=[lon.min(), lon.max(), lat.min(), lat.max()])
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')
    ax[0].set_title('GOES Day Cloud Phase RGB Composite - ' + date + ' ' + str(time_of_day) + ' UTC')
    ax[0].axis('on')  # Show the axis

    # Select and plot the timeseries of the RGB channels for the pixel defined
    # 48.464462 lat	-122.959918 lon
    pixel_lat = 48.464462
    pixel_lon = -122.959918
    selected_pixel = timeseries_ds.sel(longitude=pixel_lon, latitude=pixel_lat, method="nearest")
    ax[0].plot(selected_pixel.longitude, selected_pixel.latitude, 'ro', markersize=5, label='Cattle Point Camera')
    ax[0].legend()
    
    
    ax[1].plot(selected_pixel.t, selected_pixel.red, label='Red Channel', color='red', alpha=0.5)
    ax[1].plot(selected_pixel.t, selected_pixel.green, label='Green Channel', color='green', alpha=0.5)
    ax[1].plot(selected_pixel.t, selected_pixel.blue, label='Blue Channel', color='blue', alpha=0.5)
    current_time = datetime.strptime(f"{date} {str(time_of_day)}", "%Y%m%d %H:%M:%S")
    ax[1].axvline(x=current_time, color='black', linestyle='--', label='selected time\nfor map above')
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('RGB Value')
    ax[1].set_title('Timeseries of RGB Channels at Cattle Point (48.464462, -122.959918)')
    ax[1].legend()

    
    if gif:
        # Save the plot as a PNG file
        filename = f'./plots/goes_RGB_{date}_{time_of_day}.png'
        plt.savefig(filename)
        plt.close()
        return filename
    else:
        return rgb_plot
        plt.show()

def make_gif(ds, date):
    """
    Create a GIF from GOES data for a specific date and time range.

    Parameters:
    ds(xarray.Dataset): The dataset containing the GOES data.
    date (str): The date in 'YYYYMMDD' format.
    start_time (str): The start time in 'HHMM' format.
    end_time (str): The end time in 'HHMM' format.
    mask (bool): Whether to apply a cloud mask.
    """
    # input_file = f'/storage/cdalden/goes/goes16/RGB_composite/goes16_C02_C05_C13_RGB_colorado_{date}.nc'
    # ds = xr.open_dataset(input_file)


    start_time = datetime.strptime(f"{date}T000000", '%Y%m%dT%H%M%S')
    end_time = datetime.strptime(f"{date}T235500", '%Y%m%dT%H%M%S')

    ds = ds.sortby('t')

    # List to store the filenames of the generated plots
    filenames = []

    # Loop through every 10-minute chunk
    current_time = start_time
    while current_time <= end_time:
        time_str = current_time.strftime('%Y-%m-%dT%H:%M:%S')
        ds_i = ds.sel(t=time_str, method='nearest')
        hour_of_day = current_time.strftime('%H:%M')
        filename = plot_rgb_image_and_timeseries(ds_i, ds, date, current_time, gif=True)
        filenames.append(filename)
        current_time += timedelta(minutes=10)
        print(f'Generated RGB image for {time_str}')

    # Create the GIF
    start_time_out = start_time.strftime('%H%M')
    end_time_out =  end_time.strftime('%H%M')
    output_gif = f'./gifs/goes_RGB_{date}_{start_time_out}_{end_time_out}.gif'
    with imageio.get_writer(output_gif, mode='I', duration=0.5) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Clean up the temporary files
    import os
    for filename in filenames:
        os.remove(filename)
 

In [58]:
date = '20220805'
file = './goes17_C02_C05_C13_rgb_washington_{}.nc'.format(date)
ds = xr.open_dataset(file)
# make time monotonic increasing
ds = ds.sortby('t')
ds_i = ds.sel(t=time_of_day, method='nearest')

In [59]:
# Generate time steps from 1400 to 2330 in 5-minute intervals
start_time = "14:00:00"
end_time = "23:55:00"
time_range = pd.date_range(start_time, end_time, freq="5T")

# Loop through each time step
for time_of_day in time_range:
    print(f'Processing time step: {time_of_day}')
    
    # Combine date and time_of_day into a single datetime object
    datetime_str = f"{date} {time_of_day.strftime('%H:%M:%S')}"  # Ensure proper format
    datetime_obj = pd.to_datetime(datetime_str, format='%Y%m%d %H:%M:%S')
    
    # Select the nearest data for the current time step
    ds_i = ds.sel(t=datetime_obj, method='nearest')
    
    # Call the plotting function
    plot_rgb_image_and_timeseries(ds_i, ds, date, time_of_day.strftime("%H:%M:%S"), gif=True)

Processing time step: 2025-07-24 14:00:00
Processing time step: 2025-07-24 14:05:00
Processing time step: 2025-07-24 14:10:00
Processing time step: 2025-07-24 14:15:00
Processing time step: 2025-07-24 14:20:00
Processing time step: 2025-07-24 14:25:00
Processing time step: 2025-07-24 14:30:00
Processing time step: 2025-07-24 14:35:00
Processing time step: 2025-07-24 14:40:00
Processing time step: 2025-07-24 14:45:00
Processing time step: 2025-07-24 14:50:00
Processing time step: 2025-07-24 14:55:00
Processing time step: 2025-07-24 15:00:00
Processing time step: 2025-07-24 15:05:00
Processing time step: 2025-07-24 15:10:00
Processing time step: 2025-07-24 15:15:00
Processing time step: 2025-07-24 15:20:00
Processing time step: 2025-07-24 15:25:00
Processing time step: 2025-07-24 15:30:00
Processing time step: 2025-07-24 15:35:00
Processing time step: 2025-07-24 15:40:00
Processing time step: 2025-07-24 15:45:00
Processing time step: 2025-07-24 15:50:00
Processing time step: 2025-07-24 1

In [60]:
# start_time_out = start_time.strftime('%H:%M:%S')
# end_time_out =  end_time.strftime('%H:%M:%S')
output_gif = f'./gifs/goes_RGB_{date}_{start_time}_{end_time}.gif'
# Directory containing the GIF files
plots_dir = './plots'
# Get all .gif files in the directory
filenames = [os.path.join(plots_dir, f) for f in os.listdir(plots_dir) if f.endswith('.png')]
# Sort filenames (optional, based on your requirements)
filenames.sort()
# Combine all GIFs into one
with imageio.get_writer(output_gif, mode='I', duration=0.5) as writer:
    for filename in filenames:
        print(f"Adding {filename} to the output GIF")
        image = imageio.imread(filename)
        writer.append_data(image)

for filename in filenames:
    os.remove(filename)


Adding ./plots/goes_RGB_20220805_14:00:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:05:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:10:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:15:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:20:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:25:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:30:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:35:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:40:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:45:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:50:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_14:55:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_15:00:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_15:05:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_15:10:00.png to the output GIF
Adding ./plots/goes_RGB_20220805_15:15:0